# Current Plan
1. Create a function to calculate the distance between descriptions, use that function to naively classify new descriptions. probably looks something like a search function.
2. Grab a text classification model of hugging face and fine-tune it on the dataset.

In [ ]:
# setup requirements, which aren't loading in the venv for some reason
%pip install -r requirements.txt

## Data Preperation

### Cleaning 
The data is already pretty clean as far as non gramatical structures go. Removing all non-alphanumeric characters and lowercasing actually seems to reduce performance a bit on the embedding model, and these characters are allready stripped in the tokenization process for the naive models.

There are some sanitisation libraries around, eg. pydantic or the NLTK library but i will stick to simple here

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re

with open('data/Datas.csv') as file:
    data = pd.read_csv(file)

#over cleaning the data hurts prerformance 
# Strip non alphanumeric characters and convert to lowercase
data['Description'] = data['Description'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))
data['Description'] = data['Description'].apply(lambda x: x.lower())

# 
data['Description'] = data['Description'].apply(lambda x: re.sub(r'&amp;', 'and', x))
data['Description'] = data['Description'].apply(lambda x: re.sub(r' & ', 'and', x))

#honestly it should all just get lost in the sauce 

# Change all Categories other than Food and Entertainment to Other
#data['Category'] = ['Other' if category not in ['Food', 'Entertainment'] else category for category in data['Category']]


#save to csv 
data.to_csv('data/cleaned_data.csv', index=False)


### Augmentation/Expansion

#### Sampling and splitting
Oversampling using a target minimum number of samples doesn't seem to work atm, but I'm out of time to debug it.
Smote and AdaSynth are used for the embedded models further down, but they required some extra data to be sythesised to work well.
having never done text generation in this way, I'm following [this guide](https://python.langchain.com/docs/modules/model_io/output_parsers/types/pydantic)

In [8]:
# Split the training set into train, test and val sets
from sklearn.model_selection import train_test_split
# class must have at least 3 samples to be split
tmp = data.groupby('Category').filter(lambda x: len(x) <= 2)
data = data.groupby('Category').filter(lambda x: len(x) > 2)
train, test = train_test_split(data, test_size=0.2, stratify=data['Category'], random_state=42)
#test = pd.concat([test, tmp], axis=0)
train, val = train_test_split(train, test_size=0.3, stratify=train['Category'], random_state=77)

#### Sythesising Data using models
[using the guide](https://python.langchain.com/docs/modules/model_io/output_parsers/types/pydantic)

In [ ]:
# MiniMA 3B model - requires cloud hosting
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer

# MiniMA
tokenizer = AutoTokenizer.from_pretrained("GeneZC/MiniMA-3B", use_fast=False)
# GPU.
model = AutoModelForCausalLM.from_pretrained("GeneZC/MiniMA-3B", use_cache=True, device_map="auto", torch_dtype=torch.float16).eval()
# CPU.
#model = AutoModelForCausalLM.from_pretrained("GeneZC/MiniMA-3B", use_cache=True, device_map="cpu", torch_dtype=torch.float16).eval()$
# model.to('cuda')

prompt = "Question: Sherrie tells the truth. Vernell says Sherrie tells the truth. Alexis says Vernell lies. Michaela says Alexis tells the truth. Elanor says Michaela tells the truth. Does Elanor tell the truth?\nAnswer: No\n\nQuestion: Kristian lies. Sherrie says Kristian lies. Delbert says Sherrie lies. Jerry says Delbert tells the truth. Shalonda says Jerry tells the truth. Does Shalonda tell the truth?\nAnswer: No\n\nQuestion: Vina tells the truth. Helene says Vina lies. Kandi says Helene tells the truth. Jamey says Kandi lies. Ka says Jamey lies. Does Ka tell the truth?\nAnswer: No\n\nQuestion: Christie tells the truth. Ka says Christie tells the truth. Delbert says Ka lies. Leda says Delbert tells the truth. Lorine says Leda tells the truth. Does Lorine tell the truth?\nAnswer:"
input_ids = tokenizer([prompt]).input_ids
output_ids = model.generate(
    torch.as_tensor(input_ids).to('cuda'),
    do_sample=True,
    temperature=0.7,
    max_new_tokens=1024,
)
output_ids = output_ids[0][len(input_ids[0]):]
output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()

In [ ]:
# Define data structure for a row in the dataset.
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator


class Row(BaseModel):
    Description: str = Field(description="The description of the item")
    Category: str = Field(description="The category of the item")
    SubCategory: str = Field(description="The subcategory of the item")

def sythesise_sample(chain, samples: List[Row]):
    # And a query intented to prompt a language model to populate the data structure.
    sample_rows = "TBC"
    row_query = "generate a sample row for the dataset. \n you can use the following categories: \n" + "\n".join(data['Category'].unique()) 
    row_query = row_query + "\n you can use the following subcategories: \n" + "\n".join(data['SubCategory'].unique())
    row_query = row_query + "\n" + "here are some rows from the dataset: \n" + "\n".join(sample_rows)
    return chain.invoke({"query": row_query}) # not sure how this outputs as I haven't run it yet
    

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Row)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

sythesise_sample(chain, Row(Description="TBC", Category="TBC", SubCategory="TBC"))




In [53]:
# generate samples for classes with less than 0.1 proportion of the major class
target_proportion = 0.1
major_class_counts = data['Category'].value_counts().max() # 135
target_count = int(major_class_counts*target_proportion) # 13

new_data = pd.DataFrame(columns=['Description', 'Category', 'SubCategory'])
for category in data['Category'].unique():
    category_data = data[data['Category'] == category]
    # sythesise samples
    i = target_count - len(category_data) 
    while i > 0:
        # use list comprehension to convert dataframe to list of Row objects
        samples = data[data['Category'] == category]
        samples = [Row(Description=sample['Description'], Category=sample['Category'], SubCategory=sample['SubCategory']) for index, sample in samples.iterrows()]
        # append sythesised samples to new_data
        new_data = new_data.append(sythesise_sample(chain, samples)) # not sure if this is right tbh
        i -= 1
    

#### Sampling - NOT IN USE

In [ ]:

# Resample the training set
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

# Oversample, then undersample the training set
# def sample_strategy(df, target_proportion):
#     major_class_counts = df.value_counts().max() # 135
#     target_count = int(major_class_counts*target_proportion) # 13
#     return {k: v if v>target_count else target_count for k, v in train['Category'].value_counts().to_dict().items()}
# # Target a minimum class count of 10% of the majority class

# # targeted oversampling not working
# X_train, y_train = train[['Description']], train['Category']
# oversampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
# # oversampler = RandomOverSampler(sampling_strategy=sample_strategy(X_train,0.1), random_state=42)
# X_train, y_train = oversampler.fit_resample(train[['Description']], train['Category'])
#undersampler = RandomUnderSampler(sampling_strategy=sample_strategy(X_train,-0.5), random_state=42)
#X_train, y_train = undersampler.fit_resample(X_train, y_train)

# # join back because I don't want to refactor the notebook rn
# train = pd.concat([X_train, y_train], axis=1)   
# print(train.shape, val.shape, test.shape)
# # save data
# train.to_csv('data/train.csv', index=False)

#### Tokenization

In [12]:
# Function to measure distance 
from sklearn.feature_extraction.text import TfidfVectorizer
# have a think about whether the data needs cleaning

tfidf = TfidfVectorizer() # use token_pattern to remove punctuation if needed
tfidf.fit(train['Description']) # val set not included here 

TfidfVectorizer()

## Evaluation tools

In [3]:
from sklearn.metrics import  confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
def evaluate(Y_true, Y_pred, dataset_name="Train"):
    # evaluate using the sklearn functions
    precision = precision_score(Y_true, Y_pred, average='macro') #TP / (TP + FP)
    recall = recall_score(Y_true, Y_pred, average='macro') #TP / (TP + FN)
    f1_macro = f1_score(Y_true, Y_pred, average='macro')
    f1 = f1_score(Y_true, Y_pred, average='weighted')
    accuracy = accuracy_score(Y_true, Y_pred) #(TP + TN) / (TP + FP + FN + TN)
    
    # plot
    print(dataset_name," Precision(macro):", precision)
    print(dataset_name," Recall(macro):", recall)
    print(dataset_name," F1 Score(macro):", f1_macro)
    print(dataset_name," F1 Score(weighted):", f1)
    print(dataset_name," Accuracy:", accuracy)
    
def plot_confusion(Y_true, Y_pred, model, fig_name="confusion.png"):
    confusion = confusion_matrix(Y_true, Y_pred)
    disp = ConfusionMatrixDisplay(confusion, display_labels= model.classes_)
    disp.plot(xticks_rotation=90)
    plt.savefig('analysis/'+fig_name, bbox_inches='tight')
    plt.show()

## 1. Basic SciKitLearn Models

### Investigating cosine similarity

In [37]:
# cosine distance
from sklearn.metrics.pairwise import cosine_similarity

values = cosine_similarity(tfidf.transform(train['Description']).toarray(), tfidf.transform(train['Description']).toarray()) # dist() = x.y / (||x|| * ||y||) or sum(x * y) / (sqrt(sum(x^2)) * sqrt(sum(y^2)))
similarities = pd.DataFrame(values, index=train.index, columns=train.index)

# plot
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(similarities)

### SKLearn Models
Testing the variety of models, SVM and Naive Bayes worked the best on the raw data, while KNN and RF didn't work as well as expected.
All models suffered from the distibution issue in the datset, only predicting the most common classes (food and entertainment).

In [ ]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

# setup KNN
k = int(np.sqrt(train['Category'].count())) # sqrt(n) is a common choice for k
knn = KNeighborsClassifier(n_neighbors=k, metric='cosine')
knn.fit(tfidf.transform(train['Description']).toarray(), train['Category'])

# predict
train_accuracy = cross_val_score(knn, tfidf.transform(train['Description']).toarray(), train['Category'], cv=5, scoring='accuracy')
val_hat = knn.predict(tfidf.transform(val['Description']).toarray())

# evaluate
print("training accuracy:", train_accuracy.mean())
evaluate(val['Category'], val_hat, "Val")
plot_confusion(val['Category'], val_hat, knn, "confusion_matricies/knn.png")

In [ ]:
# SVM
from sklearn.svm import SVC

# prepare data labels to be numerical
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train['Category'])
train_Y = le.transform(train['Category'])
val_Y = le.transform(val['Category'])

# setup SVM
svm = SVC(kernel='linear')
svm.fit(tfidf.transform(train['Description']).toarray(), train_Y)

# predict
train_accuracy = cross_val_score(svm, tfidf.transform(train['Description']).toarray(), train['Category'], cv=5, scoring='accuracy')
val_hat = svm.predict(tfidf.transform(val['Description']).toarray())

# evaluate
print("training accuracy:", train_accuracy.mean())
evaluate(val_Y, val_hat, "Val")
plot_confusion(val_Y, val_hat, svm, "confusion_matricies/svm.png")

## 2. HuggingFace Models

### Embedding

In [ ]:
# MiniLM encoding; full guide: https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
import pandas as pd 
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(train['Description'].values)

assert len(embeddings) == train['Description'].count() 

### SMOTE / ADASYN synthetic oversampling
This method is really promising, and had a big effect when used on the categories with enough data, but there just isn't enough data to make it work well.

In [47]:
# SMOTE - requires more than 6 samples 
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)
train_X, train_y = smote.fit_resample(embeddings, train['Category'])

In [ ]:
# ADASYN - requires more than 6 samples 
from imblearn.over_sampling import ADASYN
adasyn = ADASYN(sampling_strategy='auto', random_state=42)
train_X, train_y = adasyn.fit_resample(embeddings, train['Category'])
train = pd.concat(train_X, train_y, axis=1)   

### Basic Models on Embeddings

In [ ]:
# KNN on embeddings
from sklearn.neighbors import KNeighborsClassifier

# setup KNN
k = int(np.sqrt(train['Category'].count())) # sqrt(n) is a common choice for k
knn = KNeighborsClassifier(n_neighbors=k, metric='cosine')
knn.fit(embeddings, train['Category']) # should still have the same indicies

# predict
train_y_hat = knn.predict(model.encode(train['Description'].values))
val_hat = knn.predict(model.encode(val['Description'].values))
test_hat = knn.predict(model.encode(test['Description'].values))

# evaluate
#evaluate(train['Category'], train_hat, "Test")
evaluate(val['Category'], val_hat, "Val")
plot_confusion(val['Category'], val_hat, knn, "confusion_matricies/knn_embeddings.png")
evaluate(test['Category'], test_hat, "Test")
plot_confusion(test['Category'], test_hat, knn, "confusion_matricies/knn_embeddings_test.png")

In [ ]:
# SVM on embeddings
from sklearn.svm import SVC

# setup SVM
svm = SVC(kernel='linear')
svm.fit(embeddings, train['Category'])

# predict
val_embeddings = model.encode(val['Description'].values)
train_hat = svm.predict(embeddings)
val_hat = svm.predict(val_embeddings)
test_hat = svm.predict(model.encode(test['Description'].values))

# Evaluate
#evaluate(train['Category'], train_hat, "Test")
evaluate(val['Category'], val_hat, "Val")
plot_confusion(val['Category'], val_hat, knn, "confusion_matricies/svm_embeddings.png")
evaluate(test['Category'], test_hat, "Test")
plot_confusion(test['Category'], test_hat, knn, "confusion_matricies/svm_embeddings_test.png")

In [ ]:
# SVM on embeddings subcategories
from sklearn.svm import SVC

# prepare data labels to be numerical
from sklearn.preprocessing import LabelEncoder
cat_le = LabelEncoder()
cat_le.fit(train['Category'])
train_cat = cat_le.transform(train['Category'])
val_cat = cat_le.transform(val['Category'])

subcat_le = LabelEncoder()
# unique values of subcategories
subcats = np.append(train['Sub_Category'].unique(), "Unknown")
subcat_le.fit(subcats)
train_Y = subcat_le.transform(train['Sub_Category'])
val['Sub_Category'] = [subcat if subcat in subcats else "Unknown" for subcat in val['Sub_Category']]
val_Y = subcat_le.transform(val['Sub_Category'])

#concat train_Y to final column in embeddings
#embeddings = np.concatenate((embeddings, train_cat), axis=1)
embeddings = np.column_stack((embeddings, train_cat))
val_embeddings = model.encode(val['Description'].values)
val_embeddings = np.column_stack((val_embeddings, val_cat))

# setup
n_svm = SVC(kernel='linear')
n_svm.fit(embeddings, train_Y)

# predict
train_y_hat = n_svm.predict(embeddings)
val_Y_hat = n_svm.predict(val_embeddings)

# Evaluate
#evaluate(train['Category'], train_hat, "Test")
evaluate(val_Y, val_Y_hat, "Val")
plot_confusion(val_Y, val_Y_hat, n_svm, "confusion_matricies/svm_embeddings_subcategories.png")

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
# SVM with class weights on embeddings
# setup
total = train['Category'].count()
class_weights = {k: 1-(v/total) for k, v in train['Category'].value_counts().to_dict().items()}
svm = SVC(kernel='linear', class_weight=class_weights)
svm.fit(embeddings, train['Category'])

# cross validate
scores = cross_val_score(svm, embeddings, train['Category'], cv=5)

# predict
train_hat = svm.predict(embeddings)
val_embeddings = model.encode(val['Description'].values)
val_hat = svm.predict(val_embeddings)
test_hat = svm.predict(model.encode(test['Description'].values))
# Evaluate
#evaluate(train['Category'], train_hat, "Test")
evaluate(val['Category'], val_hat, "Val")
plot_confusion(val['Category'], val_hat, knn, "confusion_matricies/svm_embeddings_weights.png")
evaluate(test['Category'], test_hat, "Test")
plot_confusion(test['Category'], test_hat, knn, "confusion_matricies/svm_embeddings_test_weights.png")